#yes/no xlm-roberta-large-squad2
Thanks to Isaman from Kaggle Discussion <br>
ref. https://medium.com/illuin/deep-learning-has-almost-all-the-answers-yes-no-question-answering-with-transformers-223bebb70189

In [ ]:
!pip install torch torchvision
!pip install transformers==2.5.1

     |████████████████████████████████| 499 kB 7.8 MB/s 
     |████████████████████████████████| 131 kB 14.3 MB/s 
     |████████████████████████████████| 895 kB 16.0 MB/s 
     |████████████████████████████████| 5.6 MB 17.9 MB/s 
     |████████████████████████████████| 1.2 MB 63.7 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 7.7 MB 77.1 MB/s 
     |████████████████████████████████| 138 kB 73.3 MB/s 
     |████████████████████████████████| 127 kB 90.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import re
import os

In [ ]:
# down load Thai-wiki QA
!gdown --id 14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r
# !wget https://drive.google.com/file/d/14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r
To: /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip
170MB [00:01, 96.4MB/s]


In [ ]:
!unzip -q ./AIFORTHAI-ThaiWikiQA_NSC2020.zip

!unzip -q ./AIFORTHAI-ThaiWikiQA_NSC2020/wiki-documents-nsc.zip

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

In [ ]:
model_name = "deepset/xlm-roberta-large-squad2" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm", "deepset/xlm-roberta-large-squad2", "monsoon-nlp/bert-base-thai", "roberta-base"]


In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/qahack2021/model")
model.to(device) # Send the model to the GPU if we have one
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def encode_data(tokenizer, questions, contexts, max_length):
    input_ids = []
    attention_masks = []

    for question, context in zip(questions, contexts):
        encoded_data = tokenizer.encode_plus(question, context, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

In [ ]:
# train test split
yes_no_df = pd.read_csv('./AIFORTHAI-ThaiWikiQA_NSC2020/ThaiQACorpus-DevelopmentDataset.csv')
yes_no_df = yes_no_df[yes_no_df['question_type']==2]

yes_no_df.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id
15000,15001,2,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,ไม่ใช่,NaN,NaN,265515
15001,15002,2,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,ไม่ใช่,NaN,NaN,319289
15002,15003,2,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,ไม่ใช่,NaN,NaN,319289
15003,15004,2,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,ไม่ใช่,NaN,NaN,773798
15004,15005,2,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,ไม่ใช่,NaN,NaN,358523


In [ ]:
def clean_text(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_HTML_syntax(input_text):
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

def context_and_hashtag(file_path):
    context_and_hashtag = {}
    with open(file_path, encoding="utf-8") as f:
        lines = f.readlines()
        hashtag = ""
        raw = ""
        for line in lines:
            if not line.startswith("#") and len(line.strip()) >= 2:
                line = clean_text(line)
                # line = line.replace("”", "").replace("“", "")
                line = line.strip()
                raw += line
            elif line.startswith("#"):
                 hashtag += line.strip()
    context_and_hashtag[file_path] = {"context": raw, "hashtag": hashtag}
    return context_and_hashtag
    
def preprocessing(df):
    # df["article_id"] = df["article_path"].apply(lambda x: x[-7:])
    df['context_and_hashtag'] = [context_and_hashtag(x) for x in df["article_path"]]
    df["context"] = [list(x.values())[0]['context'] for x in df['context_and_hashtag']]
    df["hashtag"] = [list(x.values())[0]['hashtag'] for x in df['context_and_hashtag']]
    df["Predicted"] = ""
    df['context'] = df['context'].map(lambda x: remove_HTML_syntax(x))
    return df

In [ ]:
text_yesno_path = './wiki-documents-nsc'

yes_no_df['article_path']  = [os.path.join(text_yesno_path, str(x)+'.txt') for x in yes_no_df['article_id']]

booleanDictionary = {'ใช่': True, 'ไม่ใช่': False}
yes_no_df['answer'] = yes_no_df['answer'].replace(booleanDictionary)

yes_no_df = preprocessing(yes_no_df)[['question_id', 'question', 'answer', 'article_id', 'article_path', 'context']]
yes_no_df.head()

,question_id,question,answer,article_id,article_path,context
15000,15001,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,False,265515,./wiki-documents-nsc/265515.txt,ภาษาฮีบรูอาซเกนาซี ภาษาฮีบรูอาซเกนาซี (Ashkena...
15001,15002,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,False,319289,./wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...
15002,15003,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,False,319289,./wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...
15003,15004,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,False,773798,./wiki-documents-nsc/773798.txt,โรคถ้ำมอง แอบถายโรคถ้ำมอง () หรือ ความชอบแอบดู...
15004,15005,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,False,358523,./wiki-documents-nsc/358523.txt,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์ เอเชียนอิน...


In [ ]:
train_yes_no, test_yes_no = train_test_split(yes_no_df, test_size=0.33, random_state=42)
display(train_yes_no)
display(test_yes_no)

,question_id,question,answer,article_id,article_path,context
15081,15082,นายแพทย์กันตพงศ์ เล่าลือพงศ์ศิริ จบการศึกษาเมื...,False,169093,./wiki-documents-nsc/169093.txt,กันตพงศ์ เล่าลือพงศ์ศิริ นายแพทย์กันตพงศ์ เล่า...
15915,15916,มูนไลต์ เป็นรายการโทรทัศน์จากสหรัฐอเมริกา ที่ส...,True,182098,./wiki-documents-nsc/182098.txt,มูนไลต์ มูนไลต์ () เป็นรายการโทรทัศน์จากสหรัฐอ...
16018,16019,มาซีโดเนียวิทยุและโทรทัศน์เป็นทรัพย์สินของทางส...,True,256700,./wiki-documents-nsc/256700.txt,สถานีวิทยุโทรทัศน์แห่งชาติมาซิโดเนีย สถานีวิทย...
15380,15381,จานบิน หรือจานผี เป็นคำทั่วไปใช้เรียกวัตถุบินท...,False,120401,./wiki-documents-nsc/120401.txt,จานบิน จานบิน () หรือจานผี เป็นวัตถุบิน ลักษณะ...
16029,16030,ถ้าความดันไอถูกผสมด้วยก๊าซอื่นเรียก ความดันย่อ...,True,23799,./wiki-documents-nsc/23799.txt,ความดันไอ ความดันไอ () คือ ความดัน (ถ้าไอถูกผส...


,question_id,question,answer,article_id,article_path,context
16860,16861,บีกานามัยซินถูกดูดซึมได้น้อยมากในทางเดินอาหาร ...,False,911752,./wiki-documents-nsc/911752.txt,บีกานามัยซิน บีกานามัยซิน หรือ กานามัยซิน บี (...
15353,15354,"""ยูบีลองวิทมี""เป็นเพลงคันทรีป็อปของนักร้องนักแ...",False,313746,./wiki-documents-nsc/313746.txt,"ยูบีลองวิทมี ""ยูบีลองวิทมี"" () เป็นเพลงแนวคันท..."
16333,16334,ลินดา หลิน ไต้เป็นนางเอกภาพยนตร์ภาษาจีนกลางในส...,True,423334,./wiki-documents-nsc/423334.txt,หลิน ไต้ ลินดา หลิน ไต้ (; ) เป็นนักแสดงหญิงชา...
15905,15906,ทรูคริปต์เป็นชุดคำสั่งที่ถูกสร้างขึ้นเพื่อใช้ใ...,True,151119,./wiki-documents-nsc/151119.txt,ทรูคริปต์ ทรูคริปต์ () เป็นซอฟต์แวร์ประเภทโอเพ...
16289,16290,มาตลีเทพบุตรถือของหอมและดอกไม้ทิพย์โปรยปรายในร...,True,58298,./wiki-documents-nsc/58298.txt,ปางเสด็จลงจากดาวดึงส์ ปางเสด็จลงจากดาวดึงส์ ทำ...


# Training

In [ ]:
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

In [ ]:
contexts_train = train_yes_no.context.values
questions_train = train_yes_no.question.values
answers_train = train_yes_no.answer.values.astype(int)

contexts_test = test_yes_no.context.values
questions_test = test_yes_no.question.values
answers_test = test_yes_no.answer.values.astype(int)

In [ ]:
# Encoding data
max_seq_length = 412
input_ids_train, attention_masks_train = encode_data(tokenizer, questions_train, contexts_train, max_seq_length)
input_ids_test, attention_masks_test = encode_data(tokenizer, questions_test, contexts_test, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
test_features = (input_ids_test, attention_masks_test, answers_test)

In [ ]:
batch_size = 4

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
test_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in test_features]

train_dataset = TensorDataset(*train_features_tensors)
test_dataset = TensorDataset(*test_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(test_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=dev_sampler, batch_size=batch_size)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache() 

In [ ]:
epochs = 5
grad_acc_steps = 1
display_steps = 1
train_loss_values = []
test_acc_values = []

In [ ]:
for epoch in range(epochs):
    # Training
    epoch_train_loss = 0 # Cumulative loss
    model.train()
    model.zero_grad()

    for step, batch in tqdm(enumerate(train_dataloader), desc='Steps: '):

        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device) 

        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

        loss = outputs[0]
        loss = loss / grad_acc_steps
        epoch_train_loss += loss.item()

        loss.backward()

        if (step+1) % grad_acc_steps == 0: # Gradient accumulation is over
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clipping gradients
            optimizer.step()
            model.zero_grad()
        # torch.cuda.empty_cache() 

    epoch_train_loss = epoch_train_loss / len(train_dataloader)          
    train_loss_values.append(epoch_train_loss)

    if epoch % display_steps == 0:
        print('Train loss: ', epoch_train_loss)

    # Evaluation
    epoch_dev_accuracy = 0 # Cumulative accuracy
    model.eval()

    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2]
                
        with torch.no_grad():        
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                    
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels = labels.numpy().flatten()

        epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

    epoch_dev_accuracy = epoch_dev_accuracy / len(test_dataloader)
    test_acc_values.append(epoch_dev_accuracy)

    if epoch % display_steps == 0:
        print('Test acc: ', epoch_dev_accuracy)

Steps: : 0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
Steps: : 335it [03:36,  1.55it/s]


Train loss:  0.4259575330115396


Steps: : 0it [00:00, ?it/s]

Test acc:  0.9363636363636364


Steps: : 335it [03:35,  1.55it/s]


...

Train loss:  0.10863958116136005


Steps: : 0it [00:00, ?it/s]

Test acc:  0.9318181818181818


Steps: : 335it [03:35,  1.56it/s]


Train loss:  0.08309530012437477
Test acc:  0.9136363636363637


# Inference

In [ ]:
def predict(question, context, max_length=416):
    sequence = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=max_length, truncation_strategy="longest_first")['input_ids'].to(device)
    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    prediction = np.array(probabilities).argmax()
    return int(prediction) # 1->Yes, 0-> NO

In [ ]:
def run_yes_no(df):
    yes_no_prediction = []
    for row in tqdm(df.itertuples()):
        question = str(row.question)
        context = str(row.context)

        if "ใช่หรือไม่" in question or "หรือไม่" in question or question in yes_no_df['question'].values:
            yes_no_prediction.append(predict(question, context))
        else:
            yes_no_prediction.append(np.nan)
    df['yes_no_prediction'] = yes_no_prediction
    return df

In [ ]:
yes_no_df['id'] = yes_no_df['question_id']
pred_yes_no = run_yes_no(yes_no_df)
pred_yes_no

2000it [01:40, 19.82it/s]


,question_id,question,answer,article_id,article_path,context,id,yes_no_prediction
15000,15001,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,False,265515,./wiki-documents-nsc/265515.txt,ภาษาฮีบรูอาซเกนาซี ภาษาฮีบรูอาซเกนาซี (Ashkena...,15001,0
15001,15002,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,False,319289,./wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...,15002,0
15002,15003,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,False,319289,./wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...,15003,0
15003,15004,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,False,773798,./wiki-documents-nsc/773798.txt,โรคถ้ำมอง แอบถายโรคถ้ำมอง () หรือ ความชอบแอบดู...,15004,0
15004,15005,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,False,358523,./wiki-documents-nsc/358523.txt,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์ เอเชียนอิน...,15005,0
...,...,...,...,...,...,...,...,...
16995,16996,ซังกริอา เป็นเครื่องดื่มประเภทหนึ่งจากสเปนและอ...,False,713240,./wiki-documents-nsc/713240.txt,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,16996,0
16996,16997,ซังกริอา ประกอบด้วยไวน์ ผลไม้หั่น สารให้ความหว...,False,713240,./wiki-documents-nsc/713240.txt,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,16997,0
16997,16998,หมู่เกาะกระเป็นหมู่เกาะเดียวของจังหวัดชุมพร ซึ...,False,396798,./wiki-documents-nsc/396798.txt,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,16998,0
16998,16999,เกาะกระ หรือ หมู่เกาะกระ ประกอบด้วย เกาะขนาดเล...,False,396798,./wiki-documents-nsc/396798.txt,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,16999,0


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred_yes_no.dropna(inplace=True)
pred_yes_no['yes_no_prediction'] = pred_yes_no['yes_no_prediction'].astype(int)
pred_yes_no["answer"] = pred_yes_no["answer"].astype(int)

print(accuracy_score(pred_yes_no['answer'], pred_yes_no['yes_no_prediction']))
print(classification_report(pred_yes_no['answer'], pred_yes_no['yes_no_prediction']))

0.9565
              precision    recall  f1-score   support

           0       0.99      0.92      0.96      1006
           1       0.93      0.99      0.96       994

    accuracy                           0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#save directory
model.save_pretrained("/content/drive/MyDrive/qahack2021/model")

# Evaluation


In [ ]:
pred_yes_no.to_csv("yesno_wiki.csv", index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/qahack2021/Test.csv")
df = df[["id", "article_id", "question"]]

In [ ]:
def id_to_path(id):
  if "SCG" in id:
    dir = "/content/drive/MyDrive/qahack2021/test"
  elif "train" in id:
    dir = "/content/drive/MyDrive/qahack2021/train"
    id = "SCG" + id[6:9]
  else:
    dir = "/content/wiki-documents-nsc"
  return os.path.join(dir, str(id)+".txt")

In [ ]:
#since the document content contain HTML syntax, we have to remove it
def remove_HTML_syntax(input_text):
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

CUTTER = 400
def cut_to_lines(text):
  length = len(text)
  if length > CUTTER:
    marker = 0
    for i in range(1,length//CUTTER+1):
      
      marker = i*CUTTER
      while marker < (i+1)*CUTTER and marker < length - 1:
        if text[marker] == " ":
          break
        # print(marker, text[marker])
        marker += 1
      text = text[:marker] + "\n" + text[marker + 1:]
  return text

# def clean_blank_branket(text):
#   return re.sub(r'\(;* *\)',"", text)

def clean_text2(text):
  text = remove_HTML_syntax(text)
  text = cut_to_lines(text)
  text = re.sub(r'\(;* *\)',"", text) #clear blank branket
  text = re.sub(r'[^A-Za-z0-9\u0E00-\u0E7F \n\u0021-\u005F\u0061-\u007E“”–]', '', text).strip()
  text = re.sub(' +', ' ', text) #clear multiple space
  return text

In [ ]:
context = []
for row in df.itertuples():
  context.append(clean_text2(open(id_to_path(row.article_id)).read()))
df["context"] = context
df

,id,article_id,question,context
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,แมว แมว เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระกูล ...
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...
...,...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,เมื่อมาทำงานจริงกับ Digitally Transforming Org...


In [ ]:
pred_df = run_yes_no(df)
pred_df.dropna(inplace=True)

In [ ]:
pred_df

,id,article_id,question,context,yes_no_prediction
1072,1073,168085,ยาคุมกำเนิดประเภทไหนที่ทำให้เกิดฝ้าใบหน้า น้ำห...,ยาคุมกำเนิดชนิดฉีด ยาฉีดคุมกำเนิด เป็นวิธีคุมก...,1.0
3660,3661,710583,ลักษณะของการเลือกที่จะพิมพ์หรือไม่พิมพ์ผลงานวิ...,ความเอนเอียงโดยการรายงาน ในวิทยาการระบาด ความเ...,1.0
5000,5001,607652,การรถไฟรัสเซียเป็นบริษัทรถไฟในประเทศเดนมาร์ก แ...,การรถไฟรัสเซีย การรถไฟรัสเซีย (JSC RZD; Rossiy...,0.0
5001,5002,271399,คาสิโอเปีย เป็นวงดนตรีร็อกญี่ปุ่นที่ก่อตั้งขึ้...,Casiopea คาสิโอเปีย เป็นวงดนตรีแจ๊สฟิวชันญี่ปุ...,0.0
5002,5003,92824,เซี่ยเหมินเป็นเมืองในมณฑลฝูเจี้ยน มีชื่อในภาษา...,เซี่ยเหมิน เซี่ยเหมิน เป็นเมืองในมณฑลฝูเจี้ยน ...,1.0
...,...,...,...,...,...
5495,5496,60306,กีฬาซีเกมส์ 1981 (พ.ศ. 2524) ใช่กีฬาที่จัดการแ...,ซีเกมส์ 1981 กีฬาซีเกมส์ 1981 (พ.ศ. 2524) ถือไ...,1.0
5496,5497,84807,ซานเซบัสเตียนเป็นเมืองรองของจังหวัดกิปุซโกอาใน...,ซานเซบัสเตียน ซานเซบัสเตียน หรือ โดโนสเตีย เป็...,1.0
5497,5498,84807,"ซานเซบัสเตียน มีประชากรประมาณ 82,930 คนใช่หรือไม่",ซานเซบัสเตียน ซานเซบัสเตียน หรือ โดโนสเตีย เป็...,0.0
5498,5499,916674,เซาท์เทิร์นครอสเกมส์ 1978 เป็นการแข่งขันกีฬาเซ...,เซาท์เทิร์นครอสเกมส์ 1978 เซาท์อเมริกันเกมส์ 1...,0.0


In [ ]:
pred_df["yes_no_prediction"] = pred_df["yes_no_prediction"].map(lambda x : "ใช่" if int(x) == 1 else "ไม่ใช่")

In [ ]:
pred_df.to_csv("yesno_test.csv")